In [8]:
import os
# Change directory to the project root
os.chdir("../") 
print(os.getcwd()) # Confirm you are now in the main project folder

c:\Users\user\Desktop\Customer_Satisfaction_Prediction_to_Production


In [9]:
%pwd

'c:\\Users\\user\\Desktop\\Customer_Satisfaction_Prediction_to_Production'

In [10]:
#4. Update the entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from customerSatisfaction.constants import *
from customerSatisfaction.utils.common import read_yaml, create_directories

In [12]:
#5. Update the configuration manager in src config


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [13]:
import os
import zipfile
import gdown
from customerSatisfaction import logger
from customerSatisfaction.utils.common import get_size

In [14]:
#6. Update the components

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [17]:
#7. Update the pipeline 

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-13 11:32:09,484: INFO: common: YAML file config\config.yaml loaded successfully]
[2026-01-13 11:32:09,515: INFO: common: YAML file params.yaml loaded successfully]
[2026-01-13 11:32:09,524: INFO: common: Created directory at: artifacts]
[2026-01-13 11:32:09,525: INFO: common: Created directory at: artifacts/data_ingestion]
[2026-01-13 11:32:09,530: INFO: 3320466772: Downloading data from https://drive.google.com/file/d/1_M8ZzFem1uUnG6d3W_BDTA9B_ewYz0SQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1_M8ZzFem1uUnG6d3W_BDTA9B_ewYz0SQ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1_M8ZzFem1uUnG6d3W_BDTA9B_ewYz0SQ&confirm=t&uuid=1dffa8a8-4df0-4d4b-8c54-f373946fda93
To: c:\Users\user\Desktop\Customer_Satisfaction_Prediction_to_Production\artifacts\data_ingestion\data.zip
100%|██████████| 44.7M/44.7M [00:46<00:00, 971kB/s] 


[2026-01-13 11:33:02,649: INFO: 3320466772: Downloaded data from https://drive.google.com/file/d/1_M8ZzFem1uUnG6d3W_BDTA9B_ewYz0SQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]


In [16]:
import os

# Get the directory where the current script is located
script_dir = os.path.dirname(os.path.abspath(__file__))
# Construct the full path to the config file
config_path = os.path.join(script_dir, 'config', 'config.yaml')

# Use config_path to open the file
with open(config_path, 'r') as file:
    # ... file operations ...
    pass


NameError: name '__file__' is not defined